# 状態空間モデル

状態空間モデルでは、未来予測を使った意思決定を、実装可能な形に分解して学びます。


## 概念の土台

状態空間モデルに入る前に、つまずきやすい用語を先にそろえます。以降のコードでは、変数がどの概念を表しているかを対応付けながら読んでください。

- **状態表現 z**: 観測を圧縮した内部表現です。予測と計画の共通基盤になります。
- **遷移モデル**: 現在状態と行動から次状態を予測するモデルです。
- **観測モデル**: 内部状態から画像やセンサ値などの観測を再構成するモデルです。
- **状態空間モデル**: 潜在状態の遷移と観測生成を分けて記述するモデルです。
- **フィルタリング**: 観測履歴から現在状態を逐次推定する操作です。

このノートでは、ここで定義した語を実験セルの変数・式に直接対応させて確認します。


## 観察 1: 状態空間モデルの遷移

状態空間モデルとして、線形遷移の最小形を定義します。


In [ ]:
z_t = 0.35
a_t = -0.3
A, B = 0.88, 0.22
z_next = A * z_t + B * a_t
print('task = state-space-models')
print('z_next =', round(z_next, 6))

この線形遷移を起点に表現学習へ接続します。

この節では、状態表現 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 観察 2: 観測予測を作る

次に、潜在状態から観測を復元する写像を作ります。状態推定と観測再現の役割分担をコードで掴みます。


In [ ]:
def decode(z):
    return {'position': 2.5 * z + 0.1, 'velocity': 0.8 * z - 0.05}
obs_next = decode(z_next)
print('obs_next =', {k: round(v, 4) for k, v in obs_next.items()})
print('keys =', list(obs_next.keys()))

観測予測を別関数に切ると、遷移誤差と観測誤差を分離して調整できます。

この節では、状態表現 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 計算の対応表

1. $z_{t+1} = f_{\theta}(z_t, a_t)$
2. $\hat{o}_{t+1} = g_{\theta}(z_{t+1})$


## 観察 3: ロールアウトを試す

ここで複数ステップ予測を実行します。1ステップでは見えない誤差累積を把握するためです。


In [ ]:
actions = [0.0, 1.0, 1.0, 0.0, -0.5]
z = 0.1
traj = []
for a in actions:
    z = 0.92 * z + 0.18 * a
    traj.append(round(z, 5))
print('rollout =', traj)

長期予測で崩れるなら、遷移モデルの安定性や状態表現の情報量不足を疑います。

この節では、状態表現 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 観察 4: 計画候補を比較する

次に、複数の行動列を比較して、どの計画が望ましいかを評価します。モデルベース強化学習の中心操作です。


In [ ]:
plans = [[0, 1, 1], [1, 1, 1], [0, 0, 1]]
def score_plan(plan):
    z = 0.1
    for a in plan:
        z = 0.92 * z + 0.18 * a
    return z
scores = [round(score_plan(p), 5) for p in plans]
print('scores =', scores)

計画評価が可能になると、実環境での試行回数を抑えた探索がしやすくなります。

この節では、状態表現 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 観察 5: モデル誤差を監視する

最後に、予測と実測の差を定量化します。世界モデルは『予測できる範囲』を常に点検する運用が重要です。


In [ ]:
pred = [0.10, 0.22, 0.31, 0.29]
real = [0.11, 0.25, 0.28, 0.35]
errors = [abs(p - r) for p, r in zip(pred, real)]
print('errors =', [round(e, 4) for e in errors])
print('mean_error =', round(sum(errors) / len(errors), 5))

平均誤差だけでなく時点別誤差を追うと、どの遷移条件でモデルが弱いかを特定しやすくなります。

この節では、状態表現 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 要点整理

今回のノートで押さえておくべき誤解しやすい点を整理します。

1. 1ステップ誤差だけで安心してしまう
2. 長期予測の誤差爆発を監視しない
3. 状態表現が不足しているのにモデルだけ調整する
